# How to Query OpenTargets Genetics for L2G Scores

### 1. Create Google Cloud Credentials

This step explains how to setup a Google Cloud Service Account to use Google BigQuery. Querying the first 1TB/month of public BigQuery is free, but there are fees beyond that, so you will need to link payment information to your credentials to run queries. Unless you are doing very large queries, you should not reach the 1TB limit.

1. Go to https://console.cloud.google.com/
2. If you do not have a project already, create one.
3. Go to https://console.cloud.google.com/iam-admin/serviceaccounts & click `+ CREATE SERVICE ACCOUNT`
4. Enter a descriptive Service Account name like "groberts-query-opentargets" & set the role to "owner"
5. Once back on the Service Accounts main page, click the "Actions" menu button > "Manage Keys"
6. `ADD KEY` > `Create New Key` > Select`JSON`.
7. A `.json` file will automatically download. **Be careful with your `.json`. You will need to call this file, but DO NOT upload it to GitHub as these are your private credentials and malicious actors could use your credentials to rack up expensive queries**

Once your service account is setup, the query below shows you how to access data in the BigQuery database
`bigquery-public-data.open_targets_genetics`.  You can view the table schema in the OpenTargets Genetics database by going to 
https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=open_targets_genetics&page=dataset .

Note that there is a separate database for the OpenTargets Platform `bigquery-public-data.open_targets_platform` that contains additional information that you may wish to map to the OpenTargets Genetics data you are pulling in, for example known drugs that target GWAS genes.

## 2. Run the Query

This demonstrates how to pull in the GWAS lead variants and genes with an L2G score of at least 0.2 across all GWAS with at least 3,000 cases and at least 10 independent genome-wide significant loci. It processes ~0.95GB of data in ~1 minute and returns 41,686 gene-GWAS connection results.

This query can be modified to be more restrictive and thereby be return results much more quickly.

In [1]:
# Setup credentials for BigQuery (.json generated in 7 steps above)

from google.cloud import bigquery
credentials_path = '/home/robertg1/.ssh/test-bigquery-ot-956f8a01208f.json' #Do not push this file to GitHub!
client = bigquery.Client.from_service_account_json(credentials_path)

In [2]:
# Basic Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Optional: Define a list of GWAS Catalog IDs/SAIGE/FINNGEN/NEAL2 GWAS IDs to keep
#           This can be useful in selecting the highest quality/latest GWAS for each trait.
study_ids_to_keep = ["GCST005523", "FINNGEN_R6_ABDOM_HERNIA"]

# Construct parameterized query
query = '''
WITH ranked_genes AS (
    SELECT locus2gene.study_id, 
        locus2gene.chrom, locus2gene.pos, locus2gene.ref, locus2gene.alt, 
        study_metadata.*, 
        genes.gene_name, locus2gene.y_proba_full_model,
        lead_variants.pval,
        ROW_NUMBER() OVER(PARTITION BY locus2gene.study_id, genes.gene_name ORDER BY lead_variants.pval) AS rn
    
    FROM `bigquery-public-data.open_targets_genetics.locus2gene` AS locus2gene
    
    -- Get GWAS metadata
    INNER JOIN `bigquery-public-data.open_targets_genetics.studies` AS study_metadata
    ON locus2gene.study_id = study_metadata.study_id
    
    -- Get HGNC IDs
    INNER JOIN `bigquery-public-data.open_targets_genetics.genes` AS genes
    ON locus2gene.gene_id = genes.gene_id
    
    -- Get lead variant P-values
    INNER JOIN `bigquery-public-data.open_targets_genetics.variant_disease` AS lead_variants
    ON locus2gene.pos = lead_variants.lead_pos
        AND locus2gene.chrom = lead_variants.lead_chrom
        AND locus2gene.study_id = lead_variants.study_id
    
        -- Optional: Filter on the study IDs in the list study_ids_to_keep
        -- WHERE locus2gene.study_id IN UNNEST(@study_ids_to_keep)
        
        -- Optional: filter on a disease
        -- AND LOWER(study_metadata.trait_reported) LIKE '%coeliac%'
        
        -- Remove the "raw" Neale lab results -- I'm not sure what this is
        AND locus2gene.study_id NOT LIKE '%raw%'
        
        -- Filter to a l2g score threshold
        AND locus2gene.y_proba_full_model > 0.2
        
        -- Filter to number of associated loci of at least 10
        AND study_metadata.num_assoc_loci >= 10
        
        -- Filter to n_cases of at least 3000
        AND study_metadata.n_cases >= 3000
)

SELECT * FROM ranked_genes WHERE rn = 1;
'''

# Set query parameters
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("study_ids_to_keep", "STRING", study_ids_to_keep)
    ]
)

# Run the query
# Note - as defined, this query returns 
query_job = client.query(query, job_config=job_config)

# Convert the query results to a Pandas dataframe
l2g = query_job.to_dataframe()
l2g.sort_values(by=['study_id', 'pval'])

,study_id,chrom,pos,ref,alt,study_id_1,ancestry_initial,ancestry_replication,n_cases,n_initial,...,has_sumstats,num_assoc_loci,source,trait_reported,trait_efos,trait_category,gene_name,y_proba_full_model,pval,rn
41253,FINNGEN_R6_ABDOM_HERNIA,1,219570796,C,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,ZC3H11B,0.685505,5.520000e-43,1
36613,FINNGEN_R6_ABDOM_HERNIA,7,134907837,G,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,CALD1,0.595480,1.000000e-15,1
41252,FINNGEN_R6_ABDOM_HERNIA,7,134907837,G,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,AGBL3,0.217192,1.000000e-15,1
36873,FINNGEN_R6_ABDOM_HERNIA,2,19576027,C,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,OSR1,0.551340,1.320000e-13,1
41357,FINNGEN_R6_ABDOM_HERNIA,12,77829818,G,A,FINNGEN_R6_ABDOM_HERNIA,{'list': [{'element': 'European=260405'}]},{'list': []},9245,260405,...,True,11,FINNGEN,Hernia of abodminal wall,{'list': [{'element': 'EFO_1001866'}]},gastrointestinal disease,NAV3,0.732569,7.730000e-11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23885,SAIGE_740_1,1,102938828,T,C,SAIGE_740_1,{'list': [{'element': 'European=398213'}]},{'list': []},17691,398213,...,True,10,SAIGE,Osteoarthritis; localized,{'list': [{'element': 'EFO_0002506'}]},Uncategorised,COL11A1,0.663457,4.860000e-10,1
22739,SAIGE_740_1,6,75478614,C,A,SAIGE_740_1,{'list': [{'element': 'European=398213'}]},{'list': []},17691,398213,...,True,10,SAIGE,Osteoarthritis; localized,{'list': [{'element': 'EFO_0002506'}]},Uncategorised,FILIP1,0.754302,4.710000e-09,1
22703,SAIGE_740_1,3,52422957,A,G,SAIGE_740_1,{'list': [{'element': 'European=398213'}]},{'list': []},17691,398213,...,True,10,SAIGE,Osteoarthritis; localized,{'list': [{'element': 'EFO_0002506'}]},Uncategorised,ITIH4,0.242676,1.420000e-08,1
22565,SAIGE_740_1,6,44991647,A,T,SAIGE_740_1,{'list': [{'element': 'European=398213'}]},{'list': []},17691,398213,...,True,10,SAIGE,Osteoarthritis; localized,{'list': [{'element': 'EFO_0002506'}]},Uncategorised,RUNX2,0.379980,2.630000e-08,1


## 3. Save the Output of the Query

In [4]:
l2g.shape

(41686, 26)

In [5]:
l2g.columns

Index(['study_id', 'chrom', 'pos', 'ref', 'alt', 'study_id_1',
       'ancestry_initial', 'ancestry_replication', 'n_cases', 'n_initial',
       'n_replication', 'pmid', 'pub_author', 'pub_date', 'pub_journal',
       'pub_title', 'has_sumstats', 'num_assoc_loci', 'source',
       'trait_reported', 'trait_efos', 'trait_category', 'gene_name',
       'y_proba_full_model', 'pval', 'rn'],
      dtype='object')

In [6]:
l2g.to_csv('gwas_data/OpenTargets_L2G_noQC.csv.gz', compression="gzip", index=False)